# Adaptive PDE discretizations on cartesian grids 
## Volume : GPU accelerated methods
## Part : Eikonal equations, acceleration and reproducibility
## Chapter : Riemannian metrics

In this notebook, we solve Riemannian eikonal equations on the CPU and the GPU, and check that they produce consistent results.

In [9]:
large_instances = False # Set to True to show off GPU acceleration (CPU times may become a big long.)

[**Summary**](Summary.ipynb) of volume GPU accelerated methods, this series of notebooks.

[**Main summary**](../Summary.ipynb) of the Adaptive Grid Discretizations 
	book of notebooks, including the other volumes.

# Table of contents
  * [1. Two dimensions](#1.-Two-dimensions)
    * [1.1 Isotropic metric](#1.1-Isotropic-metric)
    * [1.2 Smooth anisotropic metric](#1.2-Smooth-anisotropic-metric)
  * [2. Three dimensions](#2.-Three-dimensions)
    * [2.1 Smooth anisotropic metric](#2.1-Smooth-anisotropic-metric)



**Acknowledgement.** The experiments presented in these notebooks are part of ongoing research.
The author would like to acknowledge fruitful informal discussions with L. Gayraud on the 
topic of GPU coding and optimization.

Copyright Jean-Marie Mirebeau, University Paris-Sud, CNRS, University Paris-Saclay

## 0. Importing the required libraries

In [1]:
import sys; sys.path.insert(0,"..")
#from Miscellaneous import TocTools; print(TocTools.displayTOC('Riemann_Repro','GPU'))

In [2]:
import cupy as cp
import numpy as np
import itertools
from matplotlib import pyplot as plt
np.set_printoptions(edgeitems=30, linewidth=100000, formatter=dict(float=lambda x: "%5.3g" % x))

In [18]:
from agd import HFMUtils
from agd import AutomaticDifferentiation as ad
from agd import Metrics
from agd import FiniteDifferences as fd
from agd import LinearParallel as lp
import agd.AutomaticDifferentiation.cupy_generic as cugen

from Notebooks_GPU.ExportedCode.Isotropic_Repro import RunCompare

In [4]:
def ReloadPackages():
    from Miscellaneous.rreload import rreload
    global HFMUtils,ad,cugen,RunGPU,RunSmart,Metrics
    HFMUtils,ad,cugen,Metrics = rreload([HFMUtils,ad,cugen,Metrics],"../..")    
    HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

In [5]:
cp = ad.functional.decorate_module_functions(cp,cugen.set_output_dtype32) # Use float32 and int32 types in place of float64 and int64
plt = ad.functional.decorate_module_functions(plt,cugen.cupy_get_args)
HFMUtils.dictIn.RunSmart = cugen.cupy_get_args(HFMUtils.RunSmart,dtype64=True,iterables=(dict,Metrics.Base))

## 1. Two dimensions

### 1.1 Isotropic metric

In [23]:
n=4000 if large_instances else 1000
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'metric':Metrics.Riemann(cp.eye(2)),
    'seeds':cp.array([[0.5,0.5]]),
    'exportValues':1,
#    'bound_active_blocks':True,
    'traits':{
        'niter_i':24,'shape_i':(12,12), # Best
    }
})
hfmIn.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)

Casting output of function eye from float64 to float32
Casting output of function array from float64 to float32


In [24]:
_,cpuOut = RunCompare(hfmIn,check=False)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.0515134334564209 seconds,  and 86 iterations.
Post-Processing
--- gpu done, turning to cpu ---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 0.823 s.
Solver time (s). GPU : 0.0515134334564209, CPU : 1.488. Device acceleration : 28.88566923536191
Max |gpuValues-cpuValues| :  2.8908252719395122e-06


In [12]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[0,1],[0,1]],dimx=n+1,sampleBoundary=True)
X = hfmInS.Grid()
cost = np.prod(np.sin(2*np.pi*X))+1.1
hfmInS.update({
    'metric': Metrics.Riemann(cost**2*fd.as_field(cp.eye(2),X.shape[1:])), # Isotropic but non-constant metric
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [25]:
RunCompare(hfmInS,variants=True)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03346681594848633, CPU : 0.066. Device acceleration : 1.972102756999359
Max |gpuValues-cpuValues| :  4.88107854068609e-06

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03699994087219238, CPU : 0.069. Device acceleration : 1.8648678450147886
Max |gpuValues-cpuValues| :  8.07258024870805e-08

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.0395054817199707, CPU : 0.065. Device acceleration : 1.6453412835399341
Max |gpuValues-cpuValues| :  4.88107854068609e-06

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03799700736999512, CPU : 0.065. Device acceleration : 1.7106610361985557
Max |gpuValues-cpuValues| :  8.07258024870805e-08


In [27]:
hfmInS.update({
    'seeds':[[0.,1.]],
    'order':2,
    'traits':{'decreasing_macro':0,},
})

In [28]:
RunCompare(hfmInS,variants=True)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.039006710052490234, CPU : 0.066. Device acceleration : 1.6920165764091781
Max |gpuValues-cpuValues| :  4.88107854068609e-06

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.0364837646484375, CPU : 0.066. Device acceleration : 1.8090238393977416
Max |gpuValues-cpuValues| :  8.07258024870805e-08

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03802299499511719, CPU : 0.065. Device acceleration : 1.70949184850765
Max |gpuValues-cpuValues| :  4.88107854068609e-06

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03900337219238281, CPU : 0.065. Device acceleration : 1.6665225683407503
Max |gpuValues-cpuValues| :  8.07258024870805e-08


### 1.2 Smooth anisotropic metric

In [47]:
n=4000 if large_instances else 1000
hfmIn = HFMUtils.dictIn({
    'model':'Riemann2',
    'seeds':cp.array([[0.,0.]]),
    'exportValues':1,
#    'bound_active_blocks':True,
    'traits':{
        'niter_i':16,'shape_i':(8,8), # Best
    },
})
hfmIn.SetRect([[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)

Casting output of function array from float64 to float32


In [48]:
def height(x): return np.sin(x[0])*np.sin(x[1])
def surface_metric(x,z,mu=10.):
    ndim,shape = x.ndim-1,x.shape[1:]
    x_ad = ad.Dense.identity(constant=x,shape_free=(ndim,))
    tensors = lp.outer_self( z(x_ad).gradient() ) + mu**-2 * fd.as_field(cp.eye(ndim),shape)
    return Metrics.Riemann(tensors)

In [49]:
hfmIn['metric'] = surface_metric(hfmIn.Grid(),height)

Casting output of function eye from float64 to float32


In [36]:
gpuOut,cpuOut = RunCompare(hfmIn,check=False)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.18400192260742188 seconds,  and 221 iterations.
Post-Processing
--- gpu done, turning to cpu ---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 1.491 s.
Solver time (s). GPU : 0.18400192260742188, CPU : 2.62. Device acceleration : 14.23898165232715
Max |gpuValues-cpuValues| :  0.0010635152816163007


In [51]:
n=200; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)
hfmInS.update({
    'metric' : surface_metric(hfmInS.Grid(),height), 
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [53]:
RunCompare(hfmInS,variants=True,tol=1e-5)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.036499738693237305, CPU : 0.094. Device acceleration : 2.575360902992338
Max |gpuValues-cpuValues| :  2.2354527901891075e-05

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.036981821060180664, CPU : 0.09. Device acceleration : 2.4336281291703465
Max |gpuValues-cpuValues| :  3.011527211893572e-05

--- Variant with {'seedRadius': 2.0} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03198885917663574, CPU : 0.093. Device acceleration : 2.907262165445588
Max |gpuValues-cpuValues| :  2.423421811181825e-05

--- Variant with {'seedRadius': 2.0} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03000044822692871, CPU : 0.09. Device acceleration : 2.99995517797681
Max |gpuValues-cpuValues| :  3.0149078184216194e-05

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03549814224243

'\nfor fact,multip in itertools.product(factor_variants,multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip})\n'

Due to the different switching criteria of the second order scheme, we do not have bit consistency in that case. The results are nevertheless quite close. Note also that we do not deactivate the `decreasing` trait here, contrary to the isotropic case, because the scheme often does not converge without it.

**Bottom line.** Second order accuracy for anisotropic metrics on the GPU is very experimental, and not much reliable, at this stage. Further investigation is needed on the matter.

In [55]:
hfmInS.update({
    'seeds':[[0.,1.]],
    'order':2,
})

In [57]:
RunCompare(hfmInS,variants=True,check=0.1)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.0339961051940918, CPU : 0.119. Device acceleration : 3.5004009818360333
Max |gpuValues-cpuValues| :  0.08098238901310739

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.04849743843078613, CPU : 0.11499999999999999. Device acceleration : 2.3712592607158833
Max |gpuValues-cpuValues| :  0.08113533453160104

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03948044776916504, CPU : 0.11499999999999999. Device acceleration : 2.9128342381622407
Max |gpuValues-cpuValues| :  0.08095333137777927

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.055997371673583984, CPU : 0.11399999999999999. Device acceleration : 2.0358098352279983
Max |gpuValues-cpuValues| :  0.08113533453160104


'\nfor fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip},check=False)\n'

If one removes enforced monotonicity, obtaining the scheme convergence is harder, and requires setting some other parameters carefully and conservatively.
<!---
hfmInS.update({
    'order2_threshold':0.03,
    'verbosity':1,
    'traits':{'decreasing_macro':0,'order2_threshold_weighted_macro':1},
    'metric' : surface_metric(hfmInS.Grid(),height),
    'multiprecision':False,
    'tol':1e-6
})
--->

In [61]:
hfmInS.update({
    'tol':1e-6, # Tolerance for the convergence of the fixed point solver
    'order2_threshold':0.03, # Use first order scheme if second order difference is too large
    'traits':{'decreasing_macro':0}, # Do not enforce monotonicity
})

In [64]:
RunCompare(hfmInS,variants=True,check=0.15)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.040494441986083984, CPU : 0.11399999999999999. Device acceleration : 2.8152011586967016
Max |gpuValues-cpuValues| :  0.13615021364053104

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.03898048400878906, CPU : 0.11499999999999999. Device acceleration : 2.9501942555169545
Max |gpuValues-cpuValues| :  0.13615051166375491

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.03999590873718262, CPU : 0.118. Device acceleration : 2.9503017614974216
Max |gpuValues-cpuValues| :  0.13615045205911014

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.04599928855895996, CPU : 0.11499999999999999. Device acceleration : 2.5000386658719256
Max |gpuValues-cpuValues| :  0.13615051166375491


'\nfor fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip},check=False)\n'

In [ ]:
# TODO : discontinuous metric

## 2. Three dimensions

### 2.1 Smooth anisotropic metric

We generalize the two dimensional test case, although it does not much make geometrical sense anymore: we are computing geodesics in a three dimensional volume viewed as an hypersurface embedded in four dimensional Euclidean space.

In [65]:
n=200 if large_instances else 100
hfmIn = HFMUtils.dictIn({
    'model':'Riemann3',
    'seeds':cp.array([[0.,0.,0.]]),
    'exportValues':1,
#    'tol':5e-3,
#    'multiprecision':True,
#    'bound_active_blocks':True,
#    'nitermax_o':200,
#    'raiseOnNonConvergence':0,
})
hfmIn.SetRect([[-np.pi,np.pi],[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)

Casting output of function array from float64 to float32


In [66]:
def height3(x): return np.sin(x[0])*np.sin(x[1])*np.sin(x[2])

In [67]:
hfmIn['metric'] = surface_metric(hfmIn.Grid(),height3)

Casting output of function eye from float64 to float32


In [81]:
gpuOut,cpuOut = RunCompare(hfmIn,check=False)

Setting the kernel traits.
Prepating the domain data (shape,metric,...)
Preparing the problem rhs (cost, seeds,...)
Preparing the GPU kernel
Running the eikonal GPU kernel
GPU kernel eikonal ran for 0.32101917266845703 seconds,  and 82 iterations.
Post-Processing
---
Field verbosity defaults to 1
Field order defaults to 1
Field seedRadius defaults to 0
Fast marching solver completed in 50.128 s.
Max |gpuValues-cpuValues| :  0.00029890265088505785
Solver time (s). GPU : 0.32101917266845703, CPU : 75.92. Device acceleration : 236.49677796163547


In [68]:
n=20; hfmInS = hfmIn.copy() # Define a small instance for bit-consistency validation
hfmInS.SetRect([[-np.pi,np.pi],[-np.pi,np.pi],[-np.pi,np.pi]],dimx=n+1,sampleBoundary=True)
hfmInS.update({
    'metric' : surface_metric(hfmInS.Grid(),height), 
    'verbosity':0,
})

Casting output of function eye from float64 to float32


In [69]:
RunCompare(hfmInS,variants=True)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.008498430252075195, CPU : 0.038. Device acceleration : 4.471413999158367
Max |gpuValues-cpuValues| :  1.7426527081698673e-05

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.009502410888671875, CPU : 0.038. Device acceleration : 3.998985146527499
Max |gpuValues-cpuValues| :  4.1609422674060426e-07

--- Variant with {'seedRadius': 2.0} and {'multiprecision': False} ---
Solver time (s). GPU : 0.007501125335693359, CPU : 0.038. Device acceleration : 5.065906553938084
Max |gpuValues-cpuValues| :  1.3427482630223864e-05

--- Variant with {'seedRadius': 2.0} and {'multiprecision': True} ---
Solver time (s). GPU : 0.007982254028320312, CPU : 0.038. Device acceleration : 4.76056009557945
Max |gpuValues-cpuValues| :  4.477440634920171e-07

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.0085003376

'\nfor fact,multip in itertools.product(factor_variants,multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip})\n'

Due to the different switching criteria of the second order scheme, we do not have bit consistency in that case. The results are nevertheless quite close.

In [70]:
hfmInS.update({
    'seeds':[[0.,1.,1.]],
    'order':2,
})

In [72]:
RunCompare(hfmInS,variants=True,check=0.06)


--- Variant with {} and {'multiprecision': False} ---
Solver time (s). GPU : 0.010474920272827148, CPU : 0.052000000000000005. Device acceleration : 4.964238261067487
Max |gpuValues-cpuValues| :  0.057056744653570135

--- Variant with {} and {'multiprecision': True} ---
Solver time (s). GPU : 0.011478662490844727, CPU : 0.053000000000000005. Device acceleration : 4.617262685637138
Max |gpuValues-cpuValues| :  0.05705668504892536

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': False} ---
Solver time (s). GPU : 0.01098012924194336, CPU : 0.052000000000000005. Device acceleration : 4.735827680548922
Max |gpuValues-cpuValues| :  0.057056744653570135

--- Variant with {'factorizationRadius': 10, 'factorizationPointChoice': 'Key'} and {'multiprecision': True} ---
Solver time (s). GPU : 0.011481285095214844, CPU : 0.051000000000000004. Device acceleration : 4.442011462746075
Max |gpuValues-cpuValues| :  0.05705668504892536


'\nfor fact,multip in itertools.product((factor_variants[0],factor_variants[2]),multip_variants):\n    print(f"\nReproducibility with options : {fact}, {multip}")\n    RunCompare({**hfmInS,**fact,**multip},check=False)\n'